In [1]:
#Importamos librerias a utilizar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as mt

In [24]:
#Importamos los datos 
datos = pd.read_csv('datos_abiertos_covid19/200527COVID19MEXICO.csv')

#Importamos 

In [33]:
#Vemos las estructura que tienen los datos
datos.head()

,ID_REGISTRO,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,...,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,UCI
0,1ba22b,3,25,2,25,2,23/04/2020,16/04/2020,27/04/2020,1,...,2,2,1,1,1,2,2,99,1,2
1,00b59f,3,27,2,27,2,24/04/2020,20/04/2020,9999-99-99,2,...,2,2,2,2,2,2,2,1,1,1
2,157def,4,2,2,26,2,08/04/2020,08/04/2020,9999-99-99,2,...,2,2,2,2,2,2,2,99,1,2
3,170aea,4,9,2,15,1,16/04/2020,08/04/2020,29/04/2020,97,...,2,2,1,2,2,2,1,99,1,97
4,01dea4,4,9,2,9,1,20/04/2020,07/04/2020,30/04/2020,97,...,2,2,2,2,1,2,2,99,1,97


Las columnas que tienen solo números, tienen su respectivo diccionario (qué significa cada cosa), primero añado los catalogos:

In [35]:
si_no = {
        '1':'SI', 
        '2':'NO', 
        '97':'NO APLICA', 
        '98':'SE IGNORA', 
        '99':'NO ESPECIFICADO'}

sexo = {
        '1':'MUJER',
        '2':'HOMBRE',
        '99':'NO ESPECIFICADO'
        }

sector = {
        '1': 'CRUZ ROJA',
        '2': 'DIF',
        '3': 'ESTATAL',
        '4': 'IMSS',
        '5': 'IMSS-BIENESTAR',
        '6': 'ISSSTE',
        '7': 'MUNICIPAL',
        '8': 'PEMEX',
        '9': 'PRIVADA',
        '10': 'SEDENA',
        '11': 'SEMAR',
        '12': 'SSA',
        '13': 'UNIVERSITARIO',
        '99': 'NO ESPECIFICADO'
        }

paciente = {
        '1':'AMBULATORIO',
        '2':'HOSPITALIZADO',
        '99':'NO ESPECIFICADO'
        }

resultado = {
        '1':'Positivo SARS-CoV-2',
        '2':'No positivo SARS-CoV-2',
        '3':'Resultado pendiente'
        }

entidades = {
        '01': 'AGUASCALIENTES',
        '02': 'BAJA CALIFORNIA',
        '03': 'BAJA CALIFORNIA SUR',
        '04': 'CAMPECHE',
        '05': 'COAHUILA DE ZARAGOZA',
        '06': 'COLIMA',
        '07': 'CHIAPAS',
        '08': 'CHIHUAHUA',
        '09': 'CIUDAD DE MÉXICO',
        '10': 'DURANGO',
        '11': 'GUANAJUATO',
        '12': 'GUERRERO',
        '13': 'HIDALGO',
        '14': 'JALISCO',
        '15': 'MÉXICO',
        '16': 'MICHOACÁN DE OCAMPO',
        '17': 'MORELOS',
        '18': 'NAYARIT',
        '19': 'NUEVO LEÓN',
        '20': 'OAXACA',
        '21': 'PUEBLA',
        '22': 'QUERÉTARO',
        '23': 'QUINTANA ROO',
        '24': 'SAN LUIS POTOSÍ',
        '25': 'SINALOA',
        '26': 'SONORA',
        '27': 'TABASCO',
        '28': 'TAMAULIPAS',
        '29': 'TLAXCALA',
        '30': 'VERACRUZ DE IGNACIO DE LA LLAVE',
        '31': 'YUCATÁN',
        '32': 'ZACATECAS',
        '36': 'ESTADOS UNIDOS MEXICANOS',
        '97': 'NO APLICA',
        '98': 'SE IGNORA',
        '99': 'NO ESPECIFICADO'
}

Ahora creo un diccionario de que cátalogo le corresponde cada columna del dataframe:

In [36]:
columna = {
        'SECTOR': sector,                                                            
        'ENTIDAD_UM': entidades,
        'SEXO': sexo,                    
        'ENTIDAD_RES': entidades,
        'TIPO_PACIENTE': paciente,
        'RESULTADO': resultado,
        'INTUBADO':si_no,	                                                             
        'NEUMONIA':si_no,                                                          
        'EMBARAZO':si_no,	                                                              
        'DIABETES':si_no,	                                                                
        'EPOC':si_no,                                                           
        'ASMA':si_no,                                                               
        'INMUSUPR':si_no,	                                                                
        'HIPERTENSION':si_no,                                                                 
        'CARDIOVASCULAR':si_no,	                                                                 
        'OBESIDAD':si_no,                                                                  
        'RENAL_CRONICA':si_no,                                                       
        'TABAQUISMO':si_no,	                                                            
        'OTRO_CASO':si_no,	                                                                       
        'UCI':si_no                                                                  
}

In [38]:
#Ejemplo de uso
print(columna['SEXO']['2'])

print(columna['ENTIDAD_RES']['05'])

HOMBRE
COAHUILA DE ZARAGOZA
